# Create csv files for publication in wide format

### Load necessary libraries

In [1]:
import csv
import json
import urllib3  # allows to access a URL with python
import math
import os
import io
import collections
import numpy as np
import pandas as pd
import xlsxwriter

# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### User parameters

In [2]:
release = '2019.Q1.G.02' # Make sure to have the correct release here

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

wd_dir = r'../'
print('data inputs dir: ' + wd_dir)

C:\Users\L.GonzalezMorales\Documents\GitHub\FIS4SDGs\unsd\notebooks
data inputs dir: ../


## Utilities

#### Convert string to camelCase

In [3]:
def camelCase(st):
    """
    https://stackoverflow.com/questions/8347048/camelcase-every-string-any-standard-library
    
    """
    output = ''.join(x for x in st.title() if x.isalnum())
    return output[0].lower() + output[1:]

#### Disable insecure request warnings when using `urllib3`.

In [4]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#### Create array to catch errors

In [5]:
error_log = []

#### Read 'long' excel file into a DataFrame and convert all columns to string type

In [8]:
def read_excel_file(path,fileName):
    xlsx = pd.ExcelFile(path + fileName)
    x = pd.read_excel(xlsx, 'Sheet1')
    x = x.reset_index(drop=True)
    
    for c in x.columns:
        x[c] = x[c].astype(str)

    return x

In [28]:
x = read_excel_file(path, 'Data_1.5.2_VC_DSR_LSGP.xlsx')
x.columns
x.shape
x.head(3)

Index(['goalCode', 'goalDesc', 'targetCode', 'targetDesc', 'indicatorCode',
       'indicatorDesc', 'indicatorTier', 'seriesCode', 'seriesDesc',
       'seriesRelease', 'timePeriod', 'geoAreaCode', 'geoAreaName', 'level',
       'parentCode', 'parentName', 'type', 'CountryProfile', 'ISO3CD',
       'UNMember', 'X', 'Y', 'value', 'valueType', 'timeDetail', 'source',
       'footnotes', 'natureCode', 'unitsCode', 'reportingTypeCode',
       'natureDesc', 'unitsDesc', 'reportingTypeDesc'],
      dtype='object')

(8060, 33)

,goalCode,goalDesc,targetCode,targetDesc,indicatorCode,indicatorDesc,indicatorTier,seriesCode,seriesDesc,seriesRelease,...,valueType,timeDetail,source,footnotes,natureCode,unitsCode,reportingTypeCode,natureDesc,unitsDesc,reportingTypeDesc
0,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


#### Find the coverage of an (unordered) list of years

In [25]:
def year_intervals (years_list):
    """ Find the coverage of an ordered list of years"""
    
    years_list = list(map(int, years_list))
    
    years_list.sort()
    
    n = len(years_list)
    
    start_y = list()
    end_y = list()
    
    start_y.append(years_list[0])
    
    if n > 1:
        for i in range(n-1):
            if(years_list[i+1] - years_list[i]>1):
                start_y.append(years_list[i+1])
                end_y.append(years_list[i])
    
    end_y.append(years_list[n-1])
    
    interval_yy = list()
    
    for i in range(len(start_y)):

        if  end_y[i] - start_y[i]> 0 :
            interval_yy.append(str(start_y[i]) + '-' + str(end_y[i]))
        else:
            interval_yy.append(str(start_y[i]))

    
    x = ",".join(interval_yy)
    return(x)


In [26]:
year_intervals(['1995','2000', '1996', '2001','2002','2003','2004'])

'1995-1996,2000-2004'

#### Collapse footnotes of multiple records

In [72]:

def collapse_footnotes(df, key_cols, footnote_coln, year_coln):
    """ collapse footnotes for pivoting"""
    grouped_by_fn = df[key_cols +  [year_coln,footnote_coln]].groupby(key_cols + [footnote_coln])
    
    footnotes = []
    for  name, group in grouped_by_fn:
        footnote_str =  list(group[footnote_coln])
        if(len(footnote_str[0])>0):
            fn_key = group[key_cols + [footnote_coln]].drop_duplicates().to_dict('records')
            fn_key[0][footnote_coln + '_range'] = '[' + year_intervals(list(group[year_coln])) + ']'
            footnotes = footnotes + fn_key
    
    footnotes_df = pd.DataFrame(footnotes)
    
    if not footnotes_df.empty:

        footnotes = []
        grouped_by_fn_2 = footnotes_df.groupby(key_cols)
        for  name, group in grouped_by_fn_2:
            
            fn_key = group[key_cols].drop_duplicates().to_dict('records')
            group_shape = group.shape
            if group_shape[0] == 1 :
                x = group[footnote_coln].values[0]
            else:
                x = group[[footnote_coln+'_range', footnote_coln]].apply(lambda x: ': '.join(x), axis=1).values
                x = ' // '.join(map(str, x)) 
                
            fn_key[0][footnote_coln] = x
            footnotes = footnotes + fn_key
            
        
        footnotes_df = pd.DataFrame(footnotes)

    
    return(footnotes_df[key_cols + [footnote_coln]])

In [73]:
fn = collapse_footnotes(x, 
                       [
                       'indicatorCode', 
                       'seriesCode', 
                       'geoAreaCode'
                       ], 
                       'footnotes',
                       'timePeriod')

In [79]:
fn.columns
fn.shape
fn.head(15)


Index(['indicatorCode', 'seriesCode', 'geoAreaCode', 'footnotes'], dtype='object')

(620, 4)

,indicatorCode,seriesCode,geoAreaCode,footnotes
0,1.5.2,VC_DSR_LSGP,1,nan
1,1.5.2,VC_DSR_LSGP,10,nan
2,1.5.2,VC_DSR_LSGP,100,[2017]: Disclaimer: the data being submitted b...
3,1.5.2,VC_DSR_LSGP,104,Disclaimer: the data being submitted by UNISDR...
4,1.5.2,VC_DSR_LSGP,108,nan
5,1.5.2,VC_DSR_LSGP,11,nan
6,1.5.2,VC_DSR_LSGP,112,nan
7,1.5.2,VC_DSR_LSGP,116,Disclaimer: the data being submitted by UNISDR...
8,1.5.2,VC_DSR_LSGP,12,nan
9,1.5.2,VC_DSR_LSGP,120,nan


NameError: name 'nan' is not defined

#### Split DataFrame into one DataFrame for regions and another df for countries

In [36]:
def split_regions_and_countries (df):
    x_regions = df.loc[df['type']!='Country']
    x_countries = df.loc[df['type']=='Country']
    return x_regions, x_countries

In [37]:
x_r, x_c = split_regions_and_countries(read_excel_file(path, 'Data_1.5.2_VC_DSR_LSGP.xlsx'))

In [58]:
def get_column_sets(long_df):
    all_columns = long_df.columns
    
    
    series_cols = ['goalCode', 'goalDesc', 
                   'targetCode', 'targetDesc', 
                   'indicatorCode', 'indicatorDesc', 'indicatorTier', 
                   'seriesCode', 'seriesDesc', 'seriesRelease',  
                   'unitsCode', 'unitsDesc']
    
    geo_cols = ['geoAreaCode', 'geoAreaName', 
                'level', 'parentCode', 'parentName', 'type', 
                'CountryProfile', 'ISO3CD', 'UNMember', 
                'X', 'Y',]
    
    
    
    notes_cols = ['valueType', 'natureCode','natureDesc','source','footnotes','timeDetail']
    
    dimension_cols = list(long_df.columns)
    dimension_cols = [x for x in dimension_cols if x not in series_cols]   
    dimension_cols = [x for x in dimension_cols if x not in geo_cols]   
    dimension_cols = [x for x in dimension_cols if x not in notes_cols] 
    dimension_cols = [x for x in dimension_cols if x not in ('value', 'timePeriod')]
    
    key_cols = series_cols + geo_cols + dimension_cols
    
    key_cols2 = series_cols + ['geoAreaCode', 'geoAreaName'] + dimension_cols
    
    key_colsMin = ['seriesCode', 'geoAreaCode'] + dimension_cols
    
    slice_cols = series_cols + dimension_cols
    
    
    dict = {}
    
    dict['series_cols'] = series_cols
    dict['geo_cols'] = geo_cols
    dict['dimension_cols'] = dimension_cols
    dict['notes_cols'] = notes_cols
    dict['key_cols'] = key_cols
    dict['key_cols2'] = key_cols2
    dict['key_colsMin'] = key_colsMin
    dict['slice_cols'] = slice_cols
    
    return dict
    
    

In [59]:
x_r.shape
x_c.shape
get_column_sets(x_c)

(1326, 33)

(6734, 33)

{'series_cols': ['goalCode',
  'goalDesc',
  'targetCode',
  'targetDesc',
  'indicatorCode',
  'indicatorDesc',
  'indicatorTier',
  'seriesCode',
  'seriesDesc',
  'seriesRelease',
  'unitsCode',
  'unitsDesc'],
 'geo_cols': ['geoAreaCode',
  'geoAreaName',
  'level',
  'parentCode',
  'parentName',
  'type',
  'CountryProfile',
  'ISO3CD',
  'UNMember',
  'X',
  'Y'],
 'dimension_cols': ['reportingTypeCode', 'reportingTypeDesc'],
 'notes_cols': ['valueType',
  'natureCode',
  'natureDesc',
  'source',
  'footnotes',
  'timeDetail'],
 'key_cols': ['goalCode',
  'goalDesc',
  'targetCode',
  'targetDesc',
  'indicatorCode',
  'indicatorDesc',
  'indicatorTier',
  'seriesCode',
  'seriesDesc',
  'seriesRelease',
  'unitsCode',
  'unitsDesc',
  'geoAreaCode',
  'geoAreaName',
  'level',
  'parentCode',
  'parentName',
  'type',
  'CountryProfile',
  'ISO3CD',
  'UNMember',
  'X',
  'Y',
  'reportingTypeCode',
  'reportingTypeDesc'],
 'key_cols2': ['goalCode',
  'goalDesc',
  'targetCode

In [66]:
regions_footnotes_df = collapse_footnotes(x_r, get_column_sets(x_r)['key_colsMin'],'footnotes', 'timePeriod')
regions_sources_df = collapse_footnotes(x_r, get_column_sets(x_r)['key_colsMin'],'source', 'timePeriod')
regions_nature_df = collapse_footnotes(x_r, get_column_sets(x_r)['key_colsMin'],'natureDesc', 'timePeriod')

regions_footnotes_df
regions_sources_df
regions_nature_df


,seriesCode,geoAreaCode,reportingTypeCode,reportingTypeDesc,footnotes
0,VC_DSR_LSGP,1,nan,nan,nan
1,VC_DSR_LSGP,11,nan,nan,nan
2,VC_DSR_LSGP,127,nan,nan,nan
3,VC_DSR_LSGP,13,nan,nan,nan
4,VC_DSR_LSGP,135,nan,nan,nan
5,VC_DSR_LSGP,14,nan,nan,nan
6,VC_DSR_LSGP,142,nan,nan,nan
7,VC_DSR_LSGP,143,nan,nan,nan
8,VC_DSR_LSGP,145,nan,nan,nan
9,VC_DSR_LSGP,15,nan,nan,nan


,seriesCode,geoAreaCode,reportingTypeCode,reportingTypeDesc,source
0,VC_DSR_LSGP,1,nan,nan,nan
1,VC_DSR_LSGP,11,nan,nan,nan
2,VC_DSR_LSGP,127,nan,nan,nan
3,VC_DSR_LSGP,13,nan,nan,nan
4,VC_DSR_LSGP,135,nan,nan,nan
5,VC_DSR_LSGP,14,nan,nan,nan
6,VC_DSR_LSGP,142,nan,nan,nan
7,VC_DSR_LSGP,143,nan,nan,nan
8,VC_DSR_LSGP,145,nan,nan,nan
9,VC_DSR_LSGP,15,nan,nan,nan


,seriesCode,geoAreaCode,reportingTypeCode,reportingTypeDesc,natureDesc
0,VC_DSR_LSGP,1,nan,nan,nan
1,VC_DSR_LSGP,11,nan,nan,nan
2,VC_DSR_LSGP,127,nan,nan,nan
3,VC_DSR_LSGP,13,nan,nan,nan
4,VC_DSR_LSGP,135,nan,nan,nan
5,VC_DSR_LSGP,14,nan,nan,nan
6,VC_DSR_LSGP,142,nan,nan,nan
7,VC_DSR_LSGP,143,nan,nan,nan
8,VC_DSR_LSGP,145,nan,nan,nan
9,VC_DSR_LSGP,15,nan,nan,nan


## Main pivot function

In [ ]:

def pivot(f,regions_catalog, long_df_regions, geo_cols2,key_cols2,slice_cols, pivot_file_name ):    
  
    
    regions_footnotes_df = collapse_footnotes(long_df_regions, key_cols2,'Footnotes', 'Year')
    regions_sources_df = collapse_footnotes(long_df_regions, key_cols2,'Source', 'Year')
    regions_nature_df = collapse_footnotes(long_df_regions, key_cols2,'Nature_Desc', 'Year')
        
    

    #-------------------------------------------------------
    # Prepare latest year for pivoting
    #-------------------------------------------------------
    
    idx = long_df_regions.groupby(key_cols2)['Year'].transform(max) == long_df_regions['Year']
    
    latest_df = long_df_regions[key_cols2 + ['Year','Value']][idx]
    
    latest_df.rename(columns={'Year': 'Latest_Year', 'Value': 'Latest_Value'}, inplace=True)
   
    
    #-------------------------------------------------------
    # Create pivot table
    #-------------------------------------------------------
    
    pivot_table = pd.pivot_table(long_df_regions,
                                 index=key_cols2,
                                 columns = ['Year'],
                                 values = ['Value'],
                                 aggfunc = lambda x: ''.join(str(v) for v in x))
    
    pivot_table = pivot_table.replace(np.nan, '', regex=True)
    
    
    #------------------------------------------------------
    # Define new column headings (since this is multi-index)
    #------------------------------------------------------
    
    new_header = key_cols2[:] 
    
    header_elements = pivot_table.columns
    for c in header_elements:
        new_header.append(c[0]+"_"+ str(c[1]))
    
    
    pivot_table = pivot_table.reset_index()
    
    pivot_table.columns = [''.join(str(col)).strip() for col in pivot_table.columns.values]
    
    pivot_table.columns = new_header
    
    
    #-------------------------------------------------------
    # Add latest year columns to pivot table
    #-------------------------------------------------------
            
    pivot_2 = pd.merge(pivot_table, 
                       latest_df[key_cols2 +['Latest_Year','Latest_Value']], 
                       how='outer', 
                       on=key_cols2)

    #--------------------------------------------------------
    slice_key = pivot_2[slice_cols].copy()
    slice_key = slice_key.drop_duplicates()
    
    country_key = pivot_2[geo_cols2].copy()
    country_key = country_key.drop_duplicates()
    
    # Add 
    
    country_key = country_key.append(regions_catalog[geo_cols2]).drop_duplicates()
    
      
    def cartesian_product_basic(left, right):
        return (
           left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1))
    
    full_key = cartesian_product_basic(country_key,slice_key)
    
    #--------------------------------------------------------
    
    if not regions_nature_df.empty:
        pivot_2 = pd.merge(pivot_2, 
                           regions_nature_df, 
                           how='outer', 
                           on=key_cols2)
    
    
    if not regions_sources_df.empty:
        pivot_2 = pd.merge(pivot_2, 
                           regions_sources_df, 
                           how='outer', 
                           on=key_cols2)
         
    if not regions_footnotes_df.empty:
        pivot_2 = pd.merge(pivot_2, 
                           regions_footnotes_df, 
                           how='outer', 
                           on=key_cols2)
    
    
    #-------------------------------------------------------
    # Add countries without data (so they can be displayed on a map)
    #-------------------------------------------------------
    
    error_log = []
    
    try:
        
       
        pivot_2 = pd.merge(full_key, pivot_2, how='left', on=key_cols2)
        
        
        pivot_2['GeoArea_Code'] = pivot_2['GeoArea_Code'].astype('int')
        pivot_2 = pivot_2.sort_values(by=['GeoArea_Code'])
        pivot_2['GeoArea_Code'] = pivot_2['GeoArea_Code'].astype('str')

             
       
        #-------------------------------------------------------
        # Export to csv file
        #-------------------------------------------------------
        
        export_csv = pivot_2.to_csv (data_dir + pivot_file_name, 
                                     index = None, 
                                     header=True,
                                     encoding='utf-8',
                                     quoting=csv.QUOTE_NONNUMERIC)
        #------------------------------------------------------
        
        
        print('finished pivoting series ' + f )
    
    except:
        
        print('===== COULD NOT WRITE TO PIVOT for file ' + f +' =====')
        

### Read list of excel files with data for each series

In [6]:
path = wd_dir + r'data/' + release + '/' 
print("path: " + path)
files = os.listdir(path)
files[0:10]

path: ../data/2019.Q1.G.02/


['Data_1.1.1_SI_POV_DAY1.xlsx',
 'Data_1.1.1_SI_POV_EMP1.xlsx',
 'Data_1.2.1_SI_POV_NAHC.xlsx',
 'Data_1.3.1_SI_COV_BENFTS.xlsx',
 'Data_1.3.1_SI_COV_CHLD.xlsx',
 'Data_1.3.1_SI_COV_DISAB.xlsx',
 'Data_1.3.1_SI_COV_LMKT.xlsx',
 'Data_1.3.1_SI_COV_LMKTPQ.xlsx',
 'Data_1.3.1_SI_COV_MATNL.xlsx',
 'Data_1.3.1_SI_COV_PENSN.xlsx']

### For a selected file:

#### 1)  Convert content of data file into a pandas DataFrame

In [7]:
f = 'Data_1.5.2_VC_DSR_LSGP.xlsx'

xlsx = pd.ExcelFile(path + f)
x = pd.read_excel(xlsx, 'Sheet1')
x.shape
x.columns
x.head(3)




(8060, 33)

Index(['goalCode', 'goalDesc', 'targetCode', 'targetDesc', 'indicatorCode',
       'indicatorDesc', 'indicatorTier', 'seriesCode', 'seriesDesc',
       'seriesRelease', 'timePeriod', 'geoAreaCode', 'geoAreaName', 'level',
       'parentCode', 'parentName', 'type', 'CountryProfile', 'ISO3CD',
       'UNMember', 'X', 'Y', 'value', 'valueType', 'timeDetail', 'source',
       'footnotes', 'natureCode', 'unitsCode', 'reportingTypeCode',
       'natureDesc', 'unitsDesc', 'reportingTypeDesc'],
      dtype='object')

,goalCode,goalDesc,targetCode,targetDesc,indicatorCode,indicatorDesc,indicatorTier,seriesCode,seriesDesc,seriesRelease,...,valueType,timeDetail,source,footnotes,natureCode,unitsCode,reportingTypeCode,natureDesc,unitsDesc,reportingTypeDesc
0,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,End poverty in all its forms everywhere,1.5,"By 2030, build the resilience of the poor and ...",1.5.2,Direct economic loss attributed to disasters i...,1,VC_DSR_LSGP,Direct economic loss attributed to disasters r...,2019.Q1.G.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
